In [1]:
import open3d as o3d
import numpy as np
from spygrt.stream import Recording, NoMoreFrames, DualRecording
from spygrt.tracker import Tracker
from spygrt.calibration import Calibrator

In [2]:
# Files with the recording of the calibration board 
cal1 = 'E:/VISION/Youssef_Test/RS_Calibration/121622061907_2022May24111913.bag'
cal2 = 'E:/VISION/Youssef_Test/RS_Calibration/123122061443_2022May24111913.bag'
c = DualRecording(cal1,cal2)
c.start_stream()
calibs = (Calibrator(c.stream1), Calibrator(c.stream2))

In [3]:
for i in range (100):
    try:
        calibs[0].find_corners3d()
        calibs[1].find_corners3d()
    except NoMoreFrames:
        print("Reached the end of stream, used only the first " +str(i) + " frames for calibration.")
        break

#calibs[0].corners_scale()
#calibs[1].corners_scale()
calibs[0].align_cameras(calibs[1])
calibs[1].align_to_board(cal2 = calibs[0])
for cal in calibs:
    cal.write_cal()
    cal._pose = o3d.core.Tensor(np.identity(4), dtype=o3d.core.Dtype.Float32,device=cal._stream.pcd.device)
c.load_calibration()
for i in range (100):
    try:
        calibs[0].find_corners3d()
        calibs[1].find_corners3d()
    except NoMoreFrames:
        print("Reached the end of stream, used only the first " +str(i) + " frames for calibration.")
        break

#calibs[0].corners_scale()
#calibs[1].corners_scale()
calibs[0].align_cameras(calibs[1])
calibs[1].align_to_board(cal2 = calibs[0])
adj = o3d.core.Tensor(np.identity(4), dtype=o3d.core.Dtype.Float32, device=cal._pose.device)
adj[1][1]=-1
adj[0][0]=-1
calibs[0]._pose = adj@calibs[0].pose
calibs[1]._pose = adj@calibs[1].pose
for cal in calibs:
    cal.write_cal()